# Step 1: Examine Dt_Customer and Income

Section: Step 1: Examine Dt_Customer and Income

**Part of:** [marketing_campaign_082825_working.ipynb](./marketing_campaign_082825_working.ipynb)

In [1]:
# Setup and data loading
from utils import ProjectConfig, load_intermediate_results, save_intermediate_results
import pandas as pd

config = ProjectConfig()
# Load data from previous notebook
df = load_intermediate_results("data_from_02_step_0.pkl", config)


In [2]:
# Examine Dt_Customer and Income
print("Initial Data Examination:")
print("\nDt_Customer data type:", df['Dt_Customer'].dtype)
print("Income data type:", df['Income'].dtype)
print("\nMissing values:")
print(df[['Dt_Customer', 'Income']].isnull().sum())

Initial Data Examination:

Dt_Customer data type: object
Income data type: object

Missing values:
Dt_Customer     0
Income         24
dtype: int64


In [3]:
df[df['Income'].isnull()].head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
134,8996,1957,PhD,Married,NaN,2,1,11/19/12,4,230,...,8,9,0,0,0,0,0,0,0,GER
262,1994,1983,Graduation,Married,NaN,1,0,11/15/13,11,5,...,2,7,0,0,0,0,0,0,0,US
394,3769,1972,PhD,Together,NaN,1,0,3/2/14,17,25,...,3,7,0,0,0,0,0,0,0,AUS
449,5255,1986,Graduation,Single,NaN,1,0,2/20/13,19,5,...,0,1,0,0,0,0,0,0,0,AUS
525,8268,1961,PhD,Married,NaN,0,1,7/11/13,23,352,...,7,6,0,0,0,0,0,0,0,CA


In [4]:
# Cleaning Income column
df['Income'] = df['Income'].str.replace('$', '').str.replace(',', '').str.strip()
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')
df['Income'].dtype

dtype('float64')

In [5]:
# Information after cleaning the Income column
print("Number of NaN values for Income column:",df['Income'].isna().sum())

print("Basic Statistical Data for Income column:")
print(df['Income'].describe())
# Missing values will be imputed later

Number of NaN values for Income column: 24
Basic Statistical Data for Income column:
count      2216.000000
mean      52247.251354
std       25173.076661
min        1730.000000
25%       35303.000000
50%       51381.500000
75%       68522.000000
max      666666.000000
Name: Income, dtype: float64


In [6]:
# Converting Dt_Customer to datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')

In [7]:
print(df['Dt_Customer'].head(10))
df['Dt_Customer'].dtype

0   2014-06-16
1   2014-06-15
2   2014-05-13
3   2014-05-11
4   2014-04-08
5   2014-03-17
6   2014-01-29
7   2014-01-18
8   2014-01-11
9   2014-01-11
Name: Dt_Customer, dtype: datetime64[ns]


dtype('<M8[ns]')

In [8]:
# Clean Education column
print("Original Education values:",df['Education'].value_counts(dropna=False))

Original Education values: Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64


In [9]:
# Data Cleaning for Education
# The instructions emphasize cleaning the Education categories.
# The presence of 2n Cycle alongside Master suggests potential redundancy, as both likely refer
# to postgraduate education. Basic is vague and may need clarification or standardization.

# Here’s how cleansing the Education values was undertaken:
# Merge 2n Cycle with Master: Since 2n Cycle corresponds to a Master’s degree in the Bologna Process,
# combining it with Master standardizes the category to a more universally recognized term.
# Clarify Basic: Without additional context, Basic likely represents education below a Bachelor’s degree
# (e.g., high school or less). It will be renamed to something clearer: "Secondary"
# Retain Graduation and rename to Bachelor

df['Education'] = df['Education'].replace('2n Cycle', 'Master')
df['Education'] = df['Education'].replace('Graduation', 'Bachelor')
df['Education'] = df['Education'].replace('Basic', 'Secondary')

print("Cleaned Education values:",df['Education'].value_counts(dropna=False))

Cleaned Education values: Education
Bachelor     1127
Master        573
PhD           486
Secondary      54
Name: count, dtype: int64


In [10]:
# Clean Marital_Status column
print("Original Marital Status values:",df['Marital_Status'].value_counts(dropna=False))

Original Marital Status values: Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
YOLO          2
Absurd        2
Name: count, dtype: int64


In [11]:
# The cleaning process used for merging non-standard categories in the Marital Status column
# Merge (YOLO, Alone, Absurd) into a standard one (Single), reducing redundancy:
# - YOLO (2 entries): Likely a joke or informal entry implying a single. Merging into Single.
# - Alone (3 entries): Implies no partner, aligning with Single. Merging into Single.
# - Absurd (2 entries): Ambiguous, but with such low frequency, it’s likely an error or non-standard
# Retain Married, Together, Divorced, and Widow as distinct categories, as they reflect standard marital statuses.
# - Married: 864 (standard, refers to legally married individuals)
# - Together: 580 (likely refers to cohabiting partners, not legally married)
# - Divorced: 232 (standard, refers to legally divorced individuals)
# - Widow: 77 (standard, refers to individuals whose spouse has passed away)

df['Marital_Status'] = df['Marital_Status'].replace(['YOLO', 'Alone', 'Absurd'], 'Single')
print("\nCleaned Marital_Status values:",df['Marital_Status'].value_counts(dropna=False))


Cleaned Marital_Status values: Marital_Status
Married     864
Together    580
Single      487
Divorced    232
Widow        77
Name: count, dtype: int64


In [12]:
# Calculate mean and median income by Marital Status and Education
# Pivot tables for visualization
mean_pivot = df.pivot_table(values='Income', index='Marital_Status', columns='Education',
                           aggfunc='mean').round(2)
median_pivot = df.pivot_table(values='Income', index='Marital_Status', columns='Education',
                             aggfunc='median').round(2)

print("\nMean Income Pivot Table:")
print(mean_pivot)
print("\nMedian Income Pivot Table:")
print(median_pivot)


Mean Income Pivot Table:
Education       Bachelor    Master       PhD  Secondary
Marital_Status                                         
Divorced        54526.04  49972.83  53096.62    9548.00
Married         50800.26  50686.06  58138.03   21960.50
Single          51365.63  53751.08  53039.67   18238.67
Together        55758.48  49495.94  56041.42   21240.07
Widow           54976.66  56211.12  60288.08   22123.00

Median Income Pivot Table:
Education       Bachelor   Master      PhD  Secondary
Marital_Status                                       
Divorced         55635.0  49297.0  50613.5     9548.0
Married          50737.0  50017.0  57081.5    22352.0
Single           49973.5  49514.0  48918.0    16383.0
Together         53977.0  47586.5  56756.0    23179.0
Widow            58275.0  51459.5  57032.0    22123.0


In [13]:

# Save results for next notebook
save_intermediate_results(df, "data_from_03_step_1.pkl", config)
print('✓ Results saved for next notebook')

Saved: outputs\data_from_03_step_1.pkl
✓ Results saved for next notebook
